In [ ]:
from lxml import etree

In [ ]:
tree = etree.parse("20201121_rss_full_history")

In [ ]:
root = tree.getroot()

In [ ]:
for item in root.iter("item"):
    for child in item:
        print("%s - %s" % (child.tag, child.text))
    break

In [ ]:
etree.tostring(item)